参考にした書籍・Webサイト
- [検定力分析と効果量](https://qiita.com/wasabi_sugi/items/55fdac6c5b34cf281314)
- [入門統計学(栗原伸一)](https://www.amazon.co.jp/%E5%85%A5%E9%96%80-%E7%B5%B1%E8%A8%88%E5%AD%A6-%E2%88%92%E6%A4%9C%E5%AE%9A%E3%81%8B%E3%82%89%E5%A4%9A%E5%A4%89%E9%87%8F%E8%A7%A3%E6%9E%90%E3%83%BB%E5%AE%9F%E9%A8%93%E8%A8%88%E7%94%BB%E6%B3%95%E3%81%BE%E3%81%A7%E2%88%92-%E6%A0%97%E5%8E%9F-%E4%BC%B8%E4%B8%80/dp/4274068552/ref=sr_1_1?ie=UTF8&qid=1544340655&sr=8-1&keywords=%E5%85%A5%E9%96%80%E7%B5%B1%E8%A8%88%E5%AD%A6)

#### 検出力分析
検出力分析で使われる記号
- $\alpha$ : 有意水準
- $\beta$ : 第2種の誤り
- $1 - \beta$ : 検定力
- ES (Effect Size) : 効果量
- ES index : 効果量を示す指標
- N (sample size) : 標本の大きさ、データ数のこと

$\alpha$, $ES$, $N$, $1-B$のそれぞれはそれ以外の3つから計算して求められる。

#### 効果量
$t$検定の場合の効果量$ES$は以下の式でも求まる

$$
\begin{align}
d =& \frac{(m_{b}-m_{a})}{\sigma} \\
\sigma =& \sqrt{\frac{(\sigma_a^{2} + \sigma_b^{2})}{2}}
\end{align}
$$

効果量の式を変形すると

$t値=効果量\ \times sample\ size$

サンプルサイズを大きくすればするほど、$t$値が大きくなり、結果微小な意味のない差も検出してしまうことになる。

![効果量の目安](figures/cohens_d.png)
[Source: 水本・竹内(2008)研究論文における効果量の報告のために](http://www.mizumot.com/files/EffectSize_KELES31.pdf)

In [21]:
# seed 1の場合の平均値
set.seed(1)
X <- sample(1:100, 20, replace=TRUE)
Y <- sample(1:100, 20, replace=TRUE)
print(mean(X))
print(mean(Y))
print(mean(X) - mean(Y))

[1] 56
[1] 47.9
[1] 8.1


In [22]:
# seed 2
set.seed(2)
X <- sample(1:100, 20, replace=TRUE)
Y <- sample(1:100, 20, replace=TRUE)
print(mean(X))
print(mean(Y))
print(mean(X) - mean(Y))

[1] 51.55
[1] 47.6
[1] 3.95


In [23]:
# seed 3
set.seed(3)
X <- sample(1:100, 20, replace=TRUE)
Y <- sample(1:100, 20, replace=TRUE)
print(mean(X))
print(mean(Y))
print(mean(X) - mean(Y))

[1] 52.05
[1] 41.3
[1] 10.75


乱数を指定して計算するだけでも、XY差は10点ぐらいは異なってしまう。

In [24]:
# Neyman-Peasonの仮説検定
X <- rnorm(20, 60, 10)
Y <- rnorm(20, 50, 10)
t.test(X, Y, var.equal = TRUE, conf.level = 0.95)


	Two Sample t-test

data:  X and Y
t = 4.0058, df = 38, p-value = 0.0002775
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
  6.171083 18.780781
sample estimates:
mean of x mean of y 
 60.53520  48.05926 


#### 信頼区間、信頼係数
- 95％信頼区間とは
      何度もサンプリングを繰り返し、サンプルの平均を求め、95％信頼区間の推定を繰り返したときに、対称の真の平均（母平均）が、それらの信頼区間の中に収まる割合が95％である

![信頼区間](figures/confidence_interval2.png)

効果量を確認すれば、優位ではないが、意味のある差を見逃してしまうことも少なくできる

In [25]:
cohens_d <- function(x, y) {
    n_x <- length(x) - 1
    n_y <- length(y) - 1
    m_diff <- abs(mean(x) - mean(y))
    pooled_sd <- sqrt((n_x * var(x) + n_y * var(y))/(n_x + n_y))
    es <- m_diff/pooled_sd
    print(es)
}

In [26]:
# 優位ではない
X3 <- rnorm(5, 65, 15)
Y3 <- rnorm(5, 50, 15)
# 効果量は大きい
cohens_d(X3, Y3)

[1] 2.863086


In [27]:
# サンプルサイズを大きくする
X4 <- rnorm(100, 65, 15)
Y4 <- rnorm(100, 50, 15)
cohens_d(X4, Y4)

[1] 1.031733


#### 効果量の視覚化
標準偏差10,平均は100と110の分布を考える。このとき、効果量ESは
$$
ES = \frac{(110 - 100)}{10} = 1
$$
平均値の差は、ESが1なので、$SD$1個分離れていることを意味している。$SD$は10なので、平均値は10離れていることになる。

In [ ]:
curve(dnorm(x, 100, 10), 60, 160, col = "#0070B9", add = TRUE)
curve(dnorm(x, 110, 10), 60, 160, col = "#B2D6E7", add = TRUE)
abline(v = 100, add = TRUE, col = "#0070B9")
abline(v = 110, add = TRUE, col = "#B2D6E7")

#### Rで検定力分析
{stats}パッケージのpower.t.test()を用いる
- power: 検定力
- 真の平均値の差: SD = 1の時は、delta=ESとなる
- sig.level: 有意水準
- n: sample size

In [35]:
# power, delta, sig.levelを指定しているので、サンプルサイズが求まる
library(stats)
power.t.test(power = 0.8,
             delta = 0.2,
             sig.level = 0.05,
             type = "two.sample")


     Two-sample t test power calculation 

              n = 393.4067
          delta = 0.2
             sd = 1
      sig.level = 0.05
          power = 0.8
    alternative = two.sided

NOTE: n is number in *each* group


効果量を代えてみた場合。$ES$が大きくなればなるほどサンプルサイズは少数で十分ということになる

In [36]:
for (i in c(0.2, 0.5, 0.8)) {
    print(power.t.test(power = 0.8,
                       delta = i, # Effect Size
                       sig.level = 0.05,
                       type = "two.sample")$n)
}

[1] 393.4067
[1] 63.76576
[1] 25.52463


事後の検定力分析。実験ノアとに検定力がどの程度あったのかを調べるもの

In [39]:
for (i in c(0.05, 0.01, 0.001)){
    print(power.t.test(n = 50000,
                       delta = 0.0104322,
                       sig.level = i,
                       type = "two.sample")$power)
}

[1] 0.10425

[1] 0.3780888
[1] 0.177124
[1] 0.05038877
